In [28]:
import pandas as pd
import spot
import os
import re

In [29]:
tools_order = ['delag', 'rabinizer4', 'ltl2tgba', 'ltl3tela', 'ltl3tela_det', 'ltl2tgba_det']

# Functions

In [30]:
ltl_path = '/home/ter/telatko/statistics/sat23-camera-ready/random'

In [31]:
def get_ltl_table(tools_order, ltl_path , mode):
    csv = pd.read_csv(f'{ltl_path}/{mode}.csv')
    ht = {
        'tool' : [],
        'ltl_success': [],
        'ltl_fail' : []
    }
    for tool in tools_order:
        #print(tool)
        tool_df = csv.loc[csv['tool'] == tool]
        success = tool_df[tool_df.exit_code == 0]
        success_count = success.shape[0]
        failed = tool_df[tool_df.exit_code != 0]
        failed_count = failed.shape[0]
        ht['tool'].append(tool)
        ht['ltl_success'].append(success_count)
        ht['ltl_fail'].append(failed_count)
        #print(f"{tool} failed: {failed_count}, successfull: {success_count}, sum: {failed_count + success_count}")
    df = pd.DataFrame(data=ht)
    return df

In [32]:
ltl_t = get_ltl_table(tools_order, ltl_path, 'random')
ltl_t

,tool,ltl_success,ltl_fail
0,delag,335,0
1,rabinizer4,335,0
2,ltl2tgba,335,0
3,ltl3tela,335,0
4,ltl3tela_det,335,0
5,ltl2tgba_det,335,0


In [73]:
def get_data(tools_order, path):
    data = {}
    data_small = {}

    llre = re.compile('^Running \[A([1234])\].*\'(.*?)\'$')
    hnre = re.compile('^name: "(.*)"$')
    nxre = re.compile('_([TU])\\s*$')
    

    for tool in tools_order:
        csv = pd.read_csv(f'{path}/patterns-telatko-' + tool + '.csv')
        log = f'{path}/autcross/' + tool + '.log'

        ht = {
    #        'aut_id': [],
            'L0.acc': [],
            'L1.acc': [],
            'L1.time': [],
            'L1.qbfr': [],
            'L2.acc': [],
            'L2.time': [],
            'L2.qbfr': [],
            'L3.acc': [],
            'L3.time': [],
            'L3.qbfr': [],
            'L4.acc': [],
            'L4.time': [],
            'L4.qbfr': [],
        }

        #for (ix,_),_ in fs.iterrows():
        #    ht['aut_id'].append(ix)

        i = 0
        for _,r in csv.iterrows():
            ht['L' + str(i) + '.acc'].append(r['output.acc_sets'])
            if i > 0:
                ht['L' + str(i) + '.time'].append(r['time'])

            i = (i + 1) % 5

        logfd = open(log, 'r')
        for ll in logfd.readlines():
            lmatch = llre.match(ll)
            if lmatch:
                level = lmatch.group(1)
                hoa = lmatch.group(2)
                hoa = path + "/" + hoa
               
                

                hoafd = open(hoa, 'r')
                aut_seen = False
                name_seen = False
                for hl in hoafd.readlines():
                    if len(hl) > 3:
                        aut_seen = True
                    hmatch = hnre.match(hl)
                    if hmatch:
                        name_seen = True
                        xmatch = nxre.search(hmatch.group(1))
                        if xmatch:
                            ht['L' + level + '.qbfr'].append(xmatch.group(1))
                        else:
                            ht['L' + level + '.qbfr'].append('U')
                        break

                hoafd.close()

                if not aut_seen:
                    ht['L' + level + '.qbfr'].append('X')
                elif not name_seen:
                    ht['L' + level + '.qbfr'].append('U')

        logfd.close()

        #for k in ht:
        #    print(tool, k, len(ht[k]))
        data_dd = pd.DataFrame(data = ht)
        data_de = data_dd.dropna()
        data_small[tool] = data_de[(data_de['L0.acc'] <= 1)]
        data_df = data_de[(data_de['L1.qbfr'] != 'X') & (data_de['L2.qbfr'] != 'X') & (data_de['L3.qbfr'] != 'X') & (data_de['L4.qbfr'] != 'X') & (data_de['L0.acc'] > 1)]
        data[tool] = data_df
    return data, data_small

In [78]:
def get_data_eigen_directory(tools_order, path):
    data = {}
    data_small = {}

    llre = re.compile('^Running \[A([1234])\].*\'(.*?)\'$')
    hnre = re.compile('^name: "(.*)"$')
    nxre = re.compile('_([TU])\\s*$')
    

    for tool in tools_order:
        csv = pd.read_csv(f'{path}/{tool}/patterns-telatko-' + tool + '.csv')
        log = f'{path}/{tool}/autcross/' + tool + '.log'

        ht = {
    #        'aut_id': [],
            'L0.acc': [],
            'L1.acc': [],
            'L1.time': [],
            'L1.qbfr': [],
            'L2.acc': [],
            'L2.time': [],
            'L2.qbfr': [],
            'L3.acc': [],
            'L3.time': [],
            'L3.qbfr': [],
            'L4.acc': [],
            'L4.time': [],
            'L4.qbfr': [],
        }

        #for (ix,_),_ in fs.iterrows():
        #    ht['aut_id'].append(ix)

        i = 0
        for _,r in csv.iterrows():
            ht['L' + str(i) + '.acc'].append(r['output.acc_sets'])
            if i > 0:
                ht['L' + str(i) + '.time'].append(r['time'])

            i = (i + 1) % 5

        logfd = open(log, 'r')
        for ll in logfd.readlines():
            lmatch = llre.match(ll)
            if lmatch:
                level = lmatch.group(1)
                hoa = lmatch.group(2)
                hoa = path + "/" + tool + "/" + hoa
               
                

                hoafd = open(hoa, 'r')
                aut_seen = False
                name_seen = False
                for hl in hoafd.readlines():
                    if len(hl) > 3:
                        aut_seen = True
                    hmatch = hnre.match(hl)
                    if hmatch:
                        name_seen = True
                        xmatch = nxre.search(hmatch.group(1))
                        if xmatch:
                            ht['L' + level + '.qbfr'].append(xmatch.group(1))
                        else:
                            ht['L' + level + '.qbfr'].append('U')
                        break

                hoafd.close()

                if not aut_seen:
                    ht['L' + level + '.qbfr'].append('X')
                elif not name_seen:
                    ht['L' + level + '.qbfr'].append('U')

        logfd.close()

        #for k in ht:
        #    print(tool, k, len(ht[k]))
        data_dd = pd.DataFrame(data = ht)
        data_de = data_dd.dropna()
        data_small[tool] = data_de[(data_de['L0.acc'] <= 1)]
        data_df = data_de[(data_de['L1.qbfr'] != 'X') & (data_de['L2.qbfr'] != 'X') & (data_de['L3.qbfr'] != 'X') & (data_de['L4.qbfr'] != 'X') & (data_de['L0.acc'] > 1)]
        data[tool] = data_df
    return data, data_small

In [34]:
def get_table(data,tools_order):
    sumdata = {
    'tool': [],
    #'ltlcount': [],
    'autcount': [],
    'L0.acc': [],
    }
    for l in range(1, 5):
        for s in ['acc', 'time', 'ratio', 'qbft', 'qbfu']:
            sumdata['L' + str(l) + '.' + s] = []

    for tool in tools_order:
        data_df = data[tool]

        sumdata['tool'].append(tool)
        sd = data_df.sum()
        #sumdata['ltlcount'].append(ltlf_translated[tool])
        sumdata['autcount'].append(len(data_df))
        for i in range(0, 5):
            sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
            if i > 0:
                t = sd['L' + str(i) + '.time']
                #if i > 1 and i < 4:
                #    t -= sd['L' + str(i - 1) + '.time']
                sumdata['L' + str(i) + '.time'].append(round(t, 1))
                sumdata['L' + str(i) + '.ratio'].append(
                    round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
                )

                for x in ['T', 'U']:
                    sumdata['L' + str(i) + '.qbf' + x.lower()].append(len(data_df[data_df['L' + str(i) + '.qbfr'] == x]))

    #for k in sumdata:
    #    print(k, len(sumdata[k]))
    r = pd.DataFrame(data = sumdata)
    return r

In [35]:
def small_aut(data_small):
    ht = {
        "tool" : [],
        "small_aut" :[],
    }

    for tool in data_small:
        df = data_small[tool]
        print(f"tool: {tool} df: {df.shape[0]}")
        ht['tool'].append(tool)
        ht['small_aut'].append(df.shape[0])
    table = pd.DataFrame(data =ht )
    return table


In [36]:
def small_aut_table(data_small):
    ht = {
        "tool" : [],
        "small_aut" : []
    }
        
    for tool in data_small:

        df = data_small[tool]
        #print(f"tool: {tool} df: {df.shape[0]}")
        ht['tool'].append(tool)
        ht['small_aut'].append(df.shape[0])
    table = pd.DataFrame(data =ht )
    return table

In [37]:
def divide_table(ltl_df, small_df, solver_data):
    count_df = pd.merge(ltl_df, small_df, on='tool')
    count_df = pd.merge(count_df, solver_data[['tool', 'autcount']])
    response_df = solver_data[['tool', 'autcount', 'L1.qbft', 'L1.qbfu', 'L2.qbft', 'L2.qbfu', 'L3.qbft', 'L3.qbfu', 'L4.qbft', 'L4.qbfu']]
    
    level_df = solver_data.drop(['L1.qbft', 'L1.qbfu', 'L2.qbft', 'L2.qbfu', 'L3.qbft', 'L3.qbfu', 'L4.qbft', 'L4.qbfu'], axis=1)
    
    return count_df, response_df, level_df

# Random automata

# SAT22 z3 version

In [38]:
sat22_path = '/home/ter/telatko/statistics/sat23-camera-ready/sat22-z3-version/random' 

In [39]:
sat22_data, sat22_small_data = get_data(tools_order, sat22_path)

In [40]:
sat22_df = get_table(sat22_data, tools_order)
sat22_small_df = small_aut_table(sat22_small_data)

In [41]:
sat22_count_df, sat22_response_df, sat22_level_df = divide_table(ltl_t, sat22_small_df, sat22_df)

In [42]:
sat22_count_df

,tool,ltl_success,ltl_fail,small_aut,autcount
0,delag,335,0,246,89
1,rabinizer4,335,0,130,205
2,ltl2tgba,335,0,320,15
3,ltl3tela,335,0,286,49
4,ltl3tela_det,335,0,291,44
5,ltl2tgba_det,335,0,291,44


In [43]:
sat22_response_df

,tool,autcount,L1.qbft,L1.qbfu,L2.qbft,L2.qbfu,L3.qbft,L3.qbfu,L4.qbft,L4.qbfu
0,delag,89,0,89,0,89,2,87,3,86
1,rabinizer4,205,0,205,0,205,3,202,2,203
2,ltl2tgba,15,0,15,0,15,1,14,1,14
3,ltl3tela,49,0,49,0,49,1,48,1,48
4,ltl3tela_det,44,0,44,0,44,0,44,1,43
5,ltl2tgba_det,44,0,44,0,44,2,42,1,43


In [44]:
sat22_level_df

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,89,234,154,48.0,34.2,153,63.1,34.6,149,148.8,36.3,148,145.8,36.8
1,rabinizer4,205,491,293,107.1,40.3,280,108.9,43.0,270,243.8,45.0,264,205.9,46.2
2,ltl2tgba,15,32,32,7.7,0.0,31,8.0,3.1,26,39.9,18.8,26,40.1,18.8
3,ltl3tela,49,120,101,25.8,15.8,100,25.8,16.7,95,57.8,20.8,95,60.0,20.8
4,ltl3tela_det,44,97,95,22.6,2.1,95,23.1,2.1,92,58.5,5.2,92,59.3,5.2
5,ltl2tgba_det,44,96,96,23.0,0.0,96,23.5,0.0,92,99.9,4.2,92,63.9,4.2


## June 2023 version


In [45]:
june23_path = '/home/ter/telatko/statistics/sat23-camera-ready/current-z3-version/random'

In [46]:
june23_data, june23_small_data = get_data(tools_order, june23_path)

In [47]:
june23_df = get_table(june23_data, tools_order)
june23_small_df = small_aut_table(june23_small_data)

In [48]:
june23_count_df, june23_response_df, june23_level_df = divide_table(ltl_t, june23_small_df, june23_df)

In [49]:
june23_count_df

,tool,ltl_success,ltl_fail,small_aut,autcount
0,delag,335,0,246,89
1,rabinizer4,335,0,130,205
2,ltl2tgba,335,0,320,15
3,ltl3tela,335,0,286,49
4,ltl3tela_det,335,0,291,44
5,ltl2tgba_det,335,0,291,44


In [50]:
june23_response_df

,tool,autcount,L1.qbft,L1.qbfu,L2.qbft,L2.qbfu,L3.qbft,L3.qbfu,L4.qbft,L4.qbfu
0,delag,89,0,89,0,89,2,87,2,87
1,rabinizer4,205,0,205,0,205,3,202,2,203
2,ltl2tgba,15,0,15,0,15,1,14,1,14
3,ltl3tela,49,0,49,0,49,1,48,1,48
4,ltl3tela_det,44,0,44,0,44,0,44,0,44
5,ltl2tgba_det,44,0,44,0,44,2,42,1,43


In [51]:
june23_level_df

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,89,234,154,47.8,34.2,169,132.2,27.8,149,191.1,36.3,148,217.7,36.8
1,rabinizer4,205,491,293,106.6,40.3,296,109.7,39.7,267,360.4,45.6,264,270.9,46.2
2,ltl2tgba,15,32,32,7.8,0.0,31,7.9,3.1,26,69.0,18.8,26,69.9,18.8
3,ltl3tela,49,120,101,26.5,15.8,110,26.7,8.3,95,87.8,20.8,95,90.5,20.8
4,ltl3tela_det,44,97,95,23.9,2.1,97,24.6,0.0,92,53.2,5.2,92,58.1,5.2
5,ltl2tgba_det,44,96,96,23.9,0.0,96,24.6,0.0,92,162.2,4.2,92,127.9,4.2


# Patterns automata

In [57]:
tools_order = ['delag', 'rabinizer4', 'ltl2tgba', 'ltl3tela', 'ltl3tela_det']

In [58]:
ltl_path = '/home/ter/telatko/statistics/sat23/patterns'

In [59]:
ltl_t = get_ltl_table(tools_order, ltl_path, 'patterns')
ltl_t

,tool,ltl_success,ltl_fail
0,delag,343,5
1,rabinizer4,336,12
2,ltl2tgba,348,0
3,ltl3tela,330,18
4,ltl3tela_det,328,20


## SAT22 z3 version

In [60]:
sat22_path = '/home/ter/telatko/statistics/sat23/SAT22_version' 

In [61]:
sat22_data, sat22_small_data = get_data(tools_order, sat22_path)

In [62]:
sat22_df = get_table(sat22_data, tools_order)
sat22_small_df = small_aut_table(sat22_small_data)

In [63]:
sat22_count_df, sat22_response_df, sat22_level_df = divide_table(ltl_t, sat22_small_df, sat22_df)

In [64]:
sat22_count_df

,tool,ltl_success,ltl_fail,small_aut,autcount
0,delag,343,5,214,129
1,rabinizer4,336,12,102,234
2,ltl2tgba,348,0,278,70
3,ltl3tela,330,18,239,91
4,ltl3tela_det,328,20,247,81


In [65]:
sat22_response_df

,tool,autcount,L1.qbft,L1.qbfu,L2.qbft,L2.qbfu,L3.qbft,L3.qbfu,L4.qbft,L4.qbfu
0,delag,129,16,113,21,108,26,103,21,108
1,rabinizer4,234,14,220,17,217,23,211,23,211
2,ltl2tgba,70,0,70,0,70,7,63,6,64
3,ltl3tela,91,12,79,15,76,18,73,18,73
4,ltl3tela_det,81,8,73,11,70,17,64,13,68


In [66]:
sat22_level_df

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,377,787.7,27.9,395,979.0,24.5,397,7231.1,24.1,373,9078.8,28.7
1,rabinizer4,234,882,552,702.8,37.4,556,938.6,37.0,564,1254.7,36.1,543,2289.7,38.4
2,ltl2tgba,70,198,198,50.8,0.0,198,78.4,0.0,189,1232.1,4.5,189,1266.0,4.5
3,ltl3tela,91,348,332,468.9,4.6,334,558.2,4.0,326,696.6,6.3,323,1551.9,7.2
4,ltl3tela_det,81,272,272,331.5,0.0,272,415.7,0.0,272,1106.6,0.0,272,1618.7,0.0


## June 2023 z3 version

Experiments have a bit different structure (so the could run on multiple ben's simultanieously), therefore we need other data manipulation function

In [80]:
tools_order = ['delag', 'rabinizer4', 'ltl3tela', 'ltl3tela_det']

In [81]:
june23_path = '/home/ter/telatko/statistics/sat23-camera-ready/current-z3-version/patterns'

In [82]:
june23_data, june23_small_data = get_data_eigen_directory(tools_order, june23_path)

In [83]:
june23_df = get_table(june23_data, tools_order)
june23_small_df = small_aut_table(june23_small_data)

In [84]:
june23_count_df, june23_response_df, june23_level_df = divide_table(ltl_t, june23_small_df, june23_df)

In [85]:
june23_count_df

,tool,ltl_success,ltl_fail,small_aut,autcount
0,delag,343,5,214,129
1,rabinizer4,336,12,102,234
2,ltl3tela,330,18,239,91
3,ltl3tela_det,328,20,247,81


In [86]:
june23_response_df

,tool,autcount,L1.qbft,L1.qbfu,L2.qbft,L2.qbfu,L3.qbft,L3.qbfu,L4.qbft,L4.qbfu
0,delag,129,14,115,20,109,23,106,19,110
1,rabinizer4,234,13,221,15,219,22,212,22,212
2,ltl3tela,91,13,78,13,78,18,73,17,74
3,ltl3tela_det,81,9,72,9,72,13,68,12,69


In [87]:
june23_level_df

,tool,autcount,L0.acc,L1.acc,L1.time,L1.ratio,L2.acc,L2.time,L2.ratio,L3.acc,L3.time,L3.ratio,L4.acc,L4.time,L4.ratio
0,delag,129,523,377,1256.7,27.9,394,1574.9,24.7,389,6304.8,25.6,365,8724.9,30.2
1,rabinizer4,234,882,544,1129.0,38.3,548,1542.8,37.9,553,1878.1,37.3,533,3895.4,39.6
2,ltl3tela,91,348,332,915.4,4.6,339,917.3,2.6,326,1224.8,6.3,319,2830.4,8.3
3,ltl3tela_det,81,272,272,655.5,0.0,272,637.6,0.0,272,1347.7,0.0,272,2469.1,0.0
